In [3]:
import numpy as np

In [15]:
INPUT_SIZE = 13
HIDDEN_UNITS = [64, 32]
EPOCHS = 1000
LEARNING_RATE = 0.1

In [13]:
def load_data(npz_path):
    data = np.load(npz_path)
    return data['x'], data['y']

In [1]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)


In [24]:
class Layer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.bias = np.zeros((1, output_size))
        
    def forward(self, X):
        self.X = X

        self.z = np.dot(X, self.weights) + self.bias
        self.a = relu(self.z)
        
        return self.a
    
    def backward(self, grad, learning_rate):
        if grad.ndim == 1:
            grad = grad.reshape(-1, 1)
        
        delta = grad * relu_derivative(self.z)
        
        dW = np.dot(self.X.T, delta)
        db = np.sum(delta, axis=0, keepdims=True)
        
        self.weights -= learning_rate * dW
        self.bias -= learning_rate * db
        
        grad_prev = np.dot(delta, self.weights.T)
        return grad_prev

In [26]:
class OutputLayer(Layer):
    def forward(self, X):
        self.X = X
        self.z = np.dot(X, self.weights) + self.bias
        return self.z  
    
    def backward(self, grad, learning_rate):
        
        if grad.ndim == 1:
            grad = grad.reshape(-1, 1)
        
        dW = np.dot(self.X.T, grad)
        db = np.sum(grad, axis=0, keepdims=True)
        
        self.weights -= learning_rate * dW
        self.bias -= learning_rate * db
        
        grad_prev = np.dot(grad, self.weights.T)
        return grad_prev

In [7]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size):
        self.layers = []
        
        sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)):
            self.layers.append(Layer(sizes[i], sizes[i+1]))
        
        self.layers.append(OutputLayer(hidden_sizes[-1], output_size))
    
    def forward(self, X):
        a = X
        for layer in self.layers:
            a = layer.forward(a)
        return a
    
    def backward(self, X, y, learning_rate):
        y_pred = self.forward(X)
        
        m = X.shape[0]
        grad = (y_pred - y) / m
        
        for layer in reversed(self.layers):
            grad = layer.backward(grad, learning_rate)

In [11]:
def train_model(input_size, hidden_units, epochs=1000, lr=0.001):
    
    X_train, y_train = load_data("train.npz")
    X_val, y_val = load_data("val.npz")
    
   
    model = NeuralNetwork(input_size=input_size, 
                         hidden_sizes=hidden_units, 
                         output_size=1)
    
   
    for epoch in range(epochs):
        
        y_pred = model.forward(X_train)
        
        
        loss = np.mean((y_pred - y_train) ** 2)
        
        
        model.backward(X_train, y_train, lr)
        
        
        val_pred = model.forward(X_val)
        val_loss = np.mean((val_pred - y_val) ** 2)
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch} | Train Loss: {loss:.4f} | Val Loss: {val_loss:.4f}")

In [10]:
def evaluate_model(model):
    X_test, y_test = load_data("test.npz")
    y_pred = model.forward(X_test)
    
    mae = np.mean(np.abs(y_pred - y_test))
    mse = np.mean((y_pred - y_test) ** 2)
    
    print(f"\nTest Results:")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")

In [27]:
model = train_model(
                       input_size=INPUT_SIZE,
                       hidden_units=HIDDEN_UNITS,
                       epochs=EPOCHS,
                       lr=LEARNING_RATE)
    
evaluate_model(model)

ValueError: non-broadcastable output operand with shape (32,1) doesn't match the broadcast shape (32,354)